### This notebook is broken down into 6 parts:

- Initialize Network.
- Forward Propagate.
- Back Propagate Error.
- Train Network.
- Predict.
- Seeds Dataset Case Study.

These steps will provide the foundation that you need to implement the backpropagation algorithm from scratch and apply it to your own predictive modeling problems.

Source : https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

### 1. Initialize Network and Data

In [1]:
# Initialize a network function - init random weight and bias values
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [2]:
# creates a small network
from random import seed
from random import random

seed(1)
network = initialize_network(2, 1, 2)
print(network)

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}], [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]


In [3]:
# Initialization - Exemple cours : 1 hidden layer (2 neurons) and 1 ontput layer (1 neuron)
network = [[{'weights': [0.2, 0.4, -0.5, -0.4]}, {'weights': [-0.3, 0.1, 0.2, 0.2]}],
           [{'weights': [-0.3, -0.2, 0.1]}]]
print(network)

[[{'weights': [0.2, 0.4, -0.5, -0.4]}, {'weights': [-0.3, 0.1, 0.2, 0.2]}], [{'weights': [-0.3, -0.2, 0.1]}]]


In [4]:
# data row and its class
row_data = [1, 0, 1, 1]
expected = [1]

### 2. Forward Propagation

We can break forward propagation down into three parts:
- Neuron Activation (= neuron input).
- Neuron Transfer (= neuron output).
- Forward Propagation.

In [5]:
# Calculate neuron activation for an input
# activation = sum(weight_i * input_i) + bias
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [6]:
# Transfer neuron activation
# output = 1 / (1 + e^(-activation))
from math import exp
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [7]:
# Forward propagate input to a network output
def forward_propagate(network, row_data):
    inputs = row_data
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            print("Inputs : " + str(activation))
            neuron['output'] = transfer(activation)
            print("Outputs : " + str(neuron['output']))
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [8]:
# Forward propagation on : exemple cours
output = forward_propagate(network, row_data)

Inputs : -0.7
Outputs : 0.3318122278318339
Inputs : 0.10000000000000003
Outputs : 0.52497918747894
Inputs : -0.10453950584533817
Outputs : 0.47388889882398544


### 3. Back Propagate Error

This part is broken down into two sections.
- Transfer Derivative.
- Error Backpropagation.

In [9]:
# Calculate the derivative of an neuron output
# derivative = output * (1.0 - output)
def transfer_derivative(output):
    return output * (1.0 - output)

In [10]:
# Backpropagate error and store in neurons
# output_error = (expected - output) * transfer_derivative(output)
# hidden_error = (weight_k * error_j) * transfer_derivative(output)
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            print("Error : " + str(neuron['delta']))

In [11]:
# Error Backpropagation
backward_propagate_error(network, expected)

Error : 0.1311690782143445
Error : -0.008724561965433263
Error : -0.006542085064168994


### 4. Update and Train Network

This part is broken down into two sections:
- Update Weights.
- Train Network.

In [12]:
# Update network weights with error
# weight = weight + learning_rate * error * input
# weight = weight + learning_rate * error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)): 
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
				print(neuron['delta'], neuron['weights'][j])                
			neuron['weights'][-1] += l_rate * neuron['delta']
			print(neuron['delta'], neuron['weights'][-1])

In [13]:
l_rate = 0.9
print("Error \t\t\t New weight")
update_weights(network, row_data, l_rate)

Error 			 New weight
-0.008724561965433263 0.19214789423111006
-0.008724561965433263 0.4
-0.008724561965433263 -0.50785210576889
-0.008724561965433263 -0.40785210576888997
-0.006542085064168994 -0.30588787655775207
-0.006542085064168994 0.1
-0.006542085064168994 0.19411212344224793
-0.006542085064168994 0.19411212344224793
0.1311690782143445 -0.26082884634154524
0.1311690782143445 -0.13802506750700472
0.1311690782143445 0.21805217039291008


In [14]:
network

[[{'weights': [0.19214789423111006,
    0.4,
    -0.50785210576889,
    -0.40785210576888997],
   'output': 0.3318122278318339,
   'delta': -0.008724561965433263},
  {'weights': [-0.30588787655775207,
    0.1,
    0.19411212344224793,
    0.19411212344224793],
   'output': 0.52497918747894,
   'delta': -0.006542085064168994}],
 [{'weights': [-0.26082884634154524,
    -0.13802506750700472,
    0.21805217039291008],
   'output': 0.47388889882398544,
   'delta': 0.1311690782143445}]]

In [15]:
# Train a network for a fixed number of epochs (iterations)
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
		print('========================')

In [16]:
# Training a network with 2 inputs, 2 neurons hidden layer, and 2 neuron output layer
seed(1)
train_dataset = [[2.7810836,2.550537003,0],
            [1.465489372,2.362125076,0],
            [3.396561688,4.400293529,0],
            [8.675418651,-0.242068655,1],
            [7.673756466,3.508563011,1]]
n_inputs = len(train_dataset[0]) - 1
n_outputs = len(set([row[-1] for row in train_dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
n_epoch = 3
l_rate = 0.5
train_network(network, train_dataset, l_rate, n_epoch, n_outputs)

Inputs : 3.2988639183529895
Outputs : 0.9643898158763548
Inputs : 2.422484871353145
Outputs : 0.9185258960543243
Inputs : 1.4467120366053836
Outputs : 0.8094918973879515
Inputs : 1.227776976537604
Outputs : 0.7734292563511262
Error : 0.029379162360871295
Error : -0.13553299084477086
Error : 0.000525476610401129
Error : -0.0067428584755714034
0.000525476610401129 0.1350949413040863
0.000525476610401129 0.8481038607067523
0.000525476610401129 0.7640373572818145
-0.0067428584755714034 0.24569279917765538
-0.0067428584755714034 0.48683613206797244
-0.0067428584755714034 0.4461196355509524
0.029379162360871295 0.665759455212664
0.029379162360871295 0.8022161118519355
0.029379162360871295 0.10854916795467054
-0.13553299084477086 -0.03700584152097382
-0.13553299084477086 0.7735198229895619
-0.13553299084477086 0.36500057248266793
Inputs : 2.9653449544017474
Outputs : 0.9509837432440319
Inputs : 1.956147656983342
Outputs : 0.8761154354578115
Inputs : 1.4445095049393482
Outputs : 0.809152002926

In [17]:
network

[[{'weights': [-0.17844065146725996, 0.8544795070390436, 0.7324561826655119],
   'output': 0.9333589943120102,
   'delta': -0.007710300001331779},
  {'weights': [0.11214339520227799, 0.42285402018617657, 0.41466499332788637],
   'output': 0.9441187905478026,
   'delta': -0.001888947423904337}],
 [{'weights': [0.4731304813839446, 0.5738907833642061, -0.13696260294783452],
   'output': 0.7394737488816925,
   'delta': -0.14246133596070834},
  {'weights': [-0.3418120868696001, 0.5243684037876412, 0.08622627152786427],
   'output': 0.52460388277496,
   'delta': 0.11856124776867097}]]

### 5. Predict

In [18]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [19]:
# Test making predictions with the network
test_dataset = [[2.7810836,2.550537003,0],
                [1.465489372,2.362125076,0],
                [7.627531214,2.759262235,1],
                [5.332441248,2.088626775,1]]

for row in test_dataset:
    prediction = predict(network, row)
    print('\nExpected=%d, Got=%d' % (row[-1], prediction))
    print("===============")

Inputs : 2.415579414304879
Outputs : 0.9180076229492343
Inputs : 1.8050499759254128
Outputs : 0.8587625542150981
Inputs : 0.7902107005746166
Outputs : 0.6878765702913766
Inputs : 0.22274811995175414
Outputs : 0.5554579160383971

Expected=0, Got=0
Inputs : 2.48934077491253
Outputs : 0.9233911821408259
Inputs : 1.5778440317059983
Outputs : 0.8288989635428888
Inputs : 0.7756193870815511
Outputs : 0.6847352245037774
Inputs : 0.20524843097753637
Outputs : 0.5511327282954418

Expected=0, Got=0
Inputs : 1.7291275781067412
Outputs : 0.8493007939112239
Inputs : 2.4368073694948444
Outputs : 0.9195913308026076
Inputs : 0.7926124797243903
Outputs : 0.688392005193893
Inputs : 0.2781296330509767
Outputs : 0.5690876190430348

Expected=1, Got=0
Inputs : 1.5656206695520505
Outputs : 0.8271584036221025
Inputs : 1.8958472880725177
Outputs : 0.8694207982386755
Inputs : 0.7533438337129927
Outputs : 0.6799068668458155
Inputs : 0.2593903276062576
Outputs : 0.5644864149229866

Expected=1, Got=0
